# Updating and deciding

This notebook contains the simulations reported in Section 4.

### Setup

In [ ]:
# parallel computing
using Distributed
addprocs(12);

In [ ]:
# needed packages
@everywhere using Distributions, Bootstrap, Statistics, LinearAlgebra, SharedArrays
using DataFrames, HypothesisTests, DelimitedFiles, StatsBase, Colors, Gadfly

In [ ]:
# default graphics
Gadfly.push_theme(:default)
set_default_plot_size(9inch, 9inch/MathConstants.golden)

function gen_brew_colors(n) # to create your own colors, here based on one of the brewer series
    cs = distinguishable_colors(n, 
        [colorant"#66c2a5", colorant"#fc8d62", colorant"#8da0cb", colorant"#e78ac3",
            colorant"#a6d854", colorant"#ffd92f", colorant"#e5c494", colorant"#b3b3b3"],
        lchoices=Float64[58, 45, 72.5, 90],
        transform=c->deuteranopic(c, 0.1),
        cchoices=Float64[20,40],
        hchoices=[75,51,35,120,180,210,270,310]
    )
    convert(Vector{Color}, cs)
end

In [ ]:
# set parameters, define priors, etc.
@everywhere begin 
    const numb_hyp = 11
    const numb_toss = 500
    const numb_sim = 1000
    const prior = fill(Float32(1/numb_hyp), numb_hyp)
    const likelihood_heads = range(0f0, stop=1, length=numb_hyp)
    const likelihood_tails = range(1f0, stop=0, length=numb_hyp)
end

In [ ]:
@everywhere datFunc(bias) = rand(Bernoulli(bias), numb_toss)

### Update rules

In [ ]:
# Bayes' rule
@everywhere function b_upd(probs::Array{Float32,1}, dat::Array{Bool,1}, toss_num::Int64)
    if dat[toss_num] == true
        @. (probs * likelihood_heads) / $dot(probs, likelihood_heads)
    else
        @. (probs * likelihood_tails) / $dot(probs, likelihood_tails)
    end
end

In [ ]:
# EXPL
@everywhere function expl_upd(probs::Array{Float32,1}, dat::Array{Bool, 1}, toss_num::Int64, bonus::Float32=0.1)
    val::Float32 = mean(dat[1:toss_num]) * 10 + 1
    vec::Array{Float32,1} = if dat[toss_num] == true
            @. (probs * likelihood_heads) / $dot(probs, likelihood_heads)
        else
            @. (probs * likelihood_tails) / $dot(probs, likelihood_tails)
        end

    if val % 1 == .5
        vec[floor(Int64, val)] += .5*bonus
        vec[ceil(Int64, val)] += .5*bonus
    else
        vec[round(Int64, val, RoundNearestTiesAway)] += bonus
    end

    return vec / (1 + bonus)
end

In [ ]:
# Good's rule
@everywhere function good_bonus(probs::Array{Float32,1}, res::Bool, λ=2) # with λ=2, we obtain the rule L2 from Douven and Schupbach, Frontiers ...

    pE::Float32 = res == true ? dot(probs, likelihood_heads) : dot(probs, likelihood_tails)
    gb::Array{Float32,1} = res == true ? log.(likelihood_heads ./ pE) : log.(likelihood_tails ./ pE)

    function rsc(i)
        if i >= 0
            1 - exp(2λ^2 * -i^2)
        else
            -1 + exp(2λ^2 * -i^2)
        end
    end

    return map(rsc, gb)

end

@everywhere function good_upd(probs::Array{Float32,1}, dat::Array{Bool,1}, toss_num::Int64, γ::Float32=0.1)

    res::Bool = dat[toss_num]

    probvec::Array{Float32,1} = if res == true
        @. (probs * likelihood_heads) / $dot(probs, likelihood_heads)
    else
        @. (probs * likelihood_tails) / $dot(probs, likelihood_tails)
    end

    goodvec::Array{Float32,1} = probvec + γ .* (probvec .* good_bonus(probs, res))

    return goodvec / sum(goodvec)

end

In [ ]:
# Popper's rule
@everywhere function pop_bonus(probs::Array{Float32,1}, res::Bool)

    pE::Float32 = res == true ? dot(probs, likelihood_heads) : dot(probs, likelihood_tails)
    pb::Array{Float32, 1} = res == true ? (likelihood_heads .- pE) ./ (likelihood_heads .+ pE) : (likelihood_tails .- pE) ./ (likelihood_tails .+ pE)

 end

@everywhere function pop_upd(probs::Array{Float32,1}, dat::Array{Bool, 1}, toss_num::Int64, γ::Float32=0.1)

    res::Bool = dat[toss_num]

    probvec::Array{Float32,1} = if res == true
        @. (probs * likelihood_heads) / $dot(probs, likelihood_heads)
    else
        @. (probs * likelihood_tails) / $dot(probs, likelihood_tails)
    end

    popvec::Array{Float32,1} = probvec + γ .* (probvec .* pop_bonus(probs, res))

    return popvec / sum(popvec)

end

### Survival distributions

Weibull distribution

In [ ]:
weibDistr = plot(
    [x->cdf(Weibull(1, i), x) for i in 50:50:250][:],
    color=["Weibull(1, $i)" for i in 50:50:250][:],
    0, 100,
    Guide.colorkey(title="Distribution"),
    Guide.xlabel("Time"),
    Guide.ylabel("Probability of death"),
    Guide.title("CDFs of Weibull distributions"),
    Scale.color_discrete_manual(gen_brew_colors(5)...),
    style(line_width=2pt, minor_label_font_size=10pt, major_label_font_size=14pt,
            key_label_font_size=11pt, key_title_font_size=13pt,
            colorkey_swatch_shape=:square))

In [ ]:
weibInter = plot(
    [x->(1 + cdf(Weibull(1, 50), x))/2,
     x->cdf(Weibull(1, 50), x),
     x->cdf(Weibull(1, 50), x)/2,
     ],
    color=["wrong", "none", "right"],
    0, 100,
    Guide.colorkey(title="Intervention"),
    Guide.xlabel("Time"),
    Guide.ylabel("Probability of death"),
    Guide.title("Effect of intervention: Weibull(1, 50)"),
    Scale.color_discrete_manual(gen_brew_colors(3)...),
    style(line_width=2pt, minor_label_font_size=10pt, major_label_font_size=14pt,
            key_label_font_size=11pt, key_title_font_size=13pt,
            colorkey_swatch_shape=:square))

Gamma distribution

In [ ]:
gamDistr = plot(
    [x->cdf(Gamma(10, i), x) for i in [10, 12, 14, 16]][:],
    color=["Γ(10, $i)" for i in [10, 12, 14, 16]][:],
    0, 500,
    Guide.colorkey(title="Distribution"),
    Guide.xlabel("Time"),
    Guide.ylabel("Probability of death"),
    Guide.title("CDFs of Gamma distributions"),
    Scale.color_discrete_manual(gen_brew_colors(4)...),
    style(line_width=2pt, minor_label_font_size=10pt, major_label_font_size=14pt,
            key_label_font_size=11pt, key_title_font_size=13pt,
            colorkey_swatch_shape=:square))

In [ ]:
gammaInter = plot(
    [x->(1 + cdf(Gamma(10, 16), x))/2,
     x->cdf(Gamma(10, 16), x),
     x->cdf(Gamma(10, 16), x)/2,
     ],
    color=["wrong", "none", "right"],
    0, 500,
    Guide.colorkey(title="Intervention"),
    Guide.xlabel("Time"),
    Guide.ylabel("Probability of death"),
    Guide.title("Effect of intervention for Γ(10, 16)"),
    Scale.color_discrete_manual(gen_brew_colors(3)...),
    style(line_width=2pt, minor_label_font_size=10pt, major_label_font_size=14pt,
            key_label_font_size=11pt, key_title_font_size=13pt,
            colorkey_swatch_shape=:square))

### Running the simulations

In [ ]:
@everywhere const numb_agents = 200
@everywhere const numb_generations = 100

In [ ]:
# starting position: 50 Bayesians, and 50 agents per other group (EXPL, Good's rule, Popper's rule), with varying values for c (varying between 0 and 0.25)
@everywhere const numb_agents = 200
groupID = repeat(1.0:4.0, inner=div(numb_agents, 4))
population_start = vcat(fill(0, div(numb_agents, 4)), rand(Uniform(0, .25), 3*div(numb_agents, 4)))
pop_start = Array{Float32,2}(hcat(groupID, population_start));

In [ ]:
@everywhere function survWei(upds::Array{Float32,2}, # modeling probability of death, based on Weibull distribution
                             hyp::Int64,
                             a::Float64, 
                             b::Float64,
                             shape::Float64=rand(Uniform(.5, 5)), 
                             scale::Float64=rand(Uniform(50, 250)), 
                             thresh::Float64=.9)
    
    t = something(findfirst(upds .> thresh), (numb_toss, 0)) # where in the matrix with probability updates do we find the first value above thresh?
    c = t[2]
    p = t[1]
    
    # cdf(Weibull(shape, scale), p) below gives the probability of death at the relevant time

    if c == hyp
        1 - (cdf(Weibull(shape, scale), p) / a) # probability goes down if right intervention is made (which is made when the truth is assigned a probability above thresh)
    elseif c == 0
        1 - cdf(Weibull(shape, scale), numb_toss + 1) # if no intervention is made, output survival probability at last time step
    else
        (1 + (b - 1) * cdf(Weibull(shape, scale), p)) / b # probability goes down if wrong intervention is made (which happens if a false hypothesis is assigned a probabilty above thresh)
    end
end

In [ ]:
@everywhere function survGam(upds::Array{Float32,2}, # modeling probability of death, based on Gamma distribution
                             hyp::Int64,
                             a::Float64, 
                             b::Float64,
                             shape::Float64=rand(Uniform(10, 16)), 
                             scale::Float64=rand(Uniform(10, 16)), 
                             thresh::Float64=.9)
    
    t = something(findfirst(upds .> thresh), (numb_toss, 0)) # where in the matrix with probability updates do we find the first value above thresh?
    c = t[2]
    p = t[1]
    
    if c == hyp
        1 - (cdf(Gamma(shape, scale), p) / a) # the probability goes down if the right intervention is made (and the right intervention is made if the truth is assigned a probability above thresh)
    elseif c == 0
        1 - cdf(Gamma(shape, scale), numb_toss + 1) # if no intervention is made, output survival probability at last time step
    else
        (1 + (b - 1) * cdf(Gamma(shape, scale), p)) / b # the probability goes down if the wrong intervention is made (which happens if a false hypothesis is assigned a probabilty above thresh)
    end
end

In [ ]:
@everywhere function patient(rule_index::Float32, c_value::Float32, dist::Function)

    rand_hyp::Int64 = rand(1:11) # pick α hypothesis ("what's wrong with the patient")
    right = rand(Uniform(1, 10)) # effect of right intervention
    wrong = rand(Uniform(1, 10)) # effect of wrong intervention

    data::Array{Bool, 1} = datFunc((rand_hyp - 1) / (numb_hyp - 1)) # generate synthetic data for this pick (the test results for the patient)
    
    updates = Array{Float32,2}(undef, numb_toss + 1, numb_hyp) # initialize array for probabilities

    updates[1, :] = prior # set prior

    if rule_index == 1.0f0
        @fastmath @inbounds for t in 1:numb_toss # generate updates
            updates[t + 1, :] = b_upd(updates[t, :], data, t)
        end
    elseif rule_index == 2.0f0
        @fastmath @inbounds for t in 1:numb_toss # generate updates
            updates[t + 1, :] = expl_upd(updates[t, :], data, t, c_value)
        end
    elseif rule_index == 3.0f0
        @fastmath @inbounds for t in 1:numb_toss # generate updates
            updates[t + 1, :] = good_upd(updates[t, :], data, t, c_value)
        end
    else
        @fastmath @inbounds for t in 1:numb_toss # generate updates
            updates[t + 1, :] = pop_upd(updates[t, :], data, t, c_value)
        end
    end

    return dist(updates, rand_hyp, right, wrong)
end

In [ ]:
#= tests doctor on 100 patients and calculates average survival score obtained by doctor, so
average probability that patients would survive =#
@everywhere @inbounds function avScore(rule_index::Float32, c_value::Float32, dist::Function)
    tot = @distributed (+) for i in 1:100
        patient(rule_index, c_value, dist)
    end
    return tot / 100
end

In [ ]:
#= tests all doctors in a population and selects the 50 percent fittest; outputs those
as well as a copy of each of them =#
function population_upd_rep(pop::Array{Float32,2}, dist::Function)
    agent_scores = SharedArray{Float32,1}(numb_agents)
    @sync @distributed for i in 1:numb_agents
        @inbounds agent_scores[i] = avScore(pop[i, :]..., dist)
    end
    best_index = findall(agent_scores .>= Statistics.median(agent_scores))
    best = pop[best_index[1:Int(numb_agents/2)], :]
    return vcat(best, best), agent_scores
end

In [ ]:
#= runs the foregoing function for 100 generations (together with the starting population,
we have 101 generations in total), registering for each generation all relevant agent properties,
so the type the doctor belongs to, the bonus value attributed by the doctor, as well as the doctor's
fitness score =#
pop_upd_c_a = Array{Float32,3}(undef, numb_agents, 2, numb_generations + 1)
pop_upd_f = Array{Float32,2}(undef, numb_agents, numb_generations + 1)

pop_upd_c_a[:, :, 1] = pop_start

@inbounds for i in 1:numb_generations
    pop_upd_c_a[:, :, i + 1], pop_upd_f[:, i] = population_upd_rep(pop_upd_c_a[:, :, i], survWei)
end

@inbounds for i in 1:numb_agents
    pop_upd_f[i, numb_generations + 1] = avScore((pop_upd_c_a[i, 1:2, numb_generations + 1])..., survWei)
end

In [ ]:
# extracts agent type and bonus values, for easy separate storage
res_a = Array{Int32,2}(undef, numb_agents, numb_generations + 1)
res_c = Array{Float32,2}(undef, numb_agents, numb_generations + 1)

for i in 1:numb_generations + 1
     res_a[:, i], res_c[:, i] = pop_upd_c_a[:, 1, i], pop_upd_c_a[:, 2, i]
end

In [ ]:
run(`mkdir data`);

In [ ]:
# stores the relevant data for all generations
writedlm("data/weib_agent_type1.txt", res_a)
writedlm("data/weib_c_value1.txt", res_c)
writedlm("data/weib_fit1.txt", pop_upd_f)

The above allows running one simulation at a time, the relevant output of which can then be stored. The function below runs 100 simulations and stores the relevant output of each. 

In [ ]:
function sim_run(dist::Function)
    k = 1
    while k < 101
        groupID = repeat(1.0:4.0, inner=div(numb_agents, 4))
        population_start = vcat(fill(0, div(numb_agents, 4)), rand(Uniform(0, .25), 3*div(numb_agents, 4)))
        pop_start = Array{Float32,2}(hcat(groupID, population_start))
        pop_upd_c_a = Array{Float32,3}(undef, numb_agents, 2, numb_generations + 1)
        pop_upd_f = Array{Float32,2}(undef, numb_agents, numb_generations + 1)
        pop_upd_c_a[:, :, 1] = pop_start

        @inbounds for i in 1:numb_generations
            pop_upd_c_a[:, :, i + 1], pop_upd_f[:, i] = population_upd_rep(pop_upd_c_a[:, :, i], dist)
        end

        @inbounds for i in 1:numb_agents
            pop_upd_f[i, numb_generations + 1] = avScore(pop_upd_c_a[i, 1:2, numb_generations + 1]..., dist)
        end

        res_a = Array{Int32,2}(undef, numb_agents, numb_generations + 1)
        res_c = Array{Float32,2}(undef, numb_agents, numb_generations + 1)

        @inbounds for i in 1:(numb_generations + 1)
            res_a[:, i], res_c[:, i] = pop_upd_c_a[:, 1, i], pop_upd_c_a[:, 2, i]
        end

        writedlm("data/weib_agent_type$k.txt", res_a)
        writedlm("data/weib_c_value$k.txt", res_c)
        writedlm("data/weib_fit$k.txt", pop_upd_f)

        population_start = nothing
        pop_start = nothing
        pop_upd_c_a = nothing
        pop_upd_f = nothing
        res_a = nothing
        res_c = nothing
        GC.gc()

        k += 1
    end
end

In [ ]:
sim_run(survWei)

### Plot counts of types per generation for one simulation

In [ ]:
fullType = readdlm("data/weib_agent_type1.txt");

In [ ]:
ks = [ keys(sort(countmap(fullType[:,i]))) for i in 1:numb_generations + 1 ]
vls = [ values(sort(countmap(fullType[:,i]))) for i in 1:numb_generations + 1 ]

group = []
freq = []
gen = []

for i in 1:101
    append!(group, collect(ks[i]))
    append!(freq, collect(vls[i]))
    append!(gen, fill(i, length(collect(ks[i]))))
end

bar_df = convert(DataFrame, hcat(group, freq, gen))

bar_df[!, :x4] = map(bar_df[!, :x1]) do x
    if x == 1
        return "Bayes"
    elseif x == 2
        return "EXPL"
    elseif x == 3
        return "Good"
    else
        return "Popper"
    end
end

rename!(bar_df, [Symbol("$i") for i in ["Group", "Count", "Generation", "Rule"]]);

In [ ]:
plot(bar_df, x=:Generation, y=:Count, color=:Rule, Geom.bar(position=:stack),
    Coord.cartesian(xmin=1, xmax=numb_generations + 1),
    Guide.colorkey(title="Rule"),
    Scale.color_discrete_manual(gen_brew_colors(4)...),
    style(minor_label_font_size=10pt, major_label_font_size=14pt,
            key_label_font_size=11pt, key_title_font_size=13pt,
            grid_color=colorant"#222831",
            colorkey_swatch_shape=:square))

### Visualize full results

In [ ]:
full_results_type = Array{Int32, 3}(undef, 200, 101, 100)

for i in 1:100
    full_results_type[:, :, i] = readdlm("data/weib_agent_type$i.txt")
end

In [ ]:
percentages = Array{Float64, 3}(undef, 100, 101, 4)

for k in 1:4
    for j in 1:101
        percentages[:, j, k] = [ length(findall(full_results_type[:, j, i] .== k)) / 200 for i in 1:100 ]
    end
end

In [ ]:
@everywhere function bs(x, n=1000)
    bootstrap(mean, x, BasicSampling(n))
end

bayesBoot = SharedArray{Float64}(101, 3);
    
@distributed for i in 1:101
    bayesBoot[i, :] = [Bootstrap.confint(bs(percentages[:, i, 1]), BasicConfInt(.95))[1]...]
end

explBoot = SharedArray{Float64}(101, 3);
    
@distributed for i in 1:101
    explBoot[i, :] = [Bootstrap.confint(bs(percentages[:, i, 2]), BasicConfInt(.95))[1]...]
end

goodBoot = SharedArray{Float64}(101, 3);
    
@distributed for i in 1:101
    goodBoot[i, :] = [Bootstrap.confint(bs(percentages[:, i, 3]), BasicConfInt(.95))[1]...]
end

popperBoot = SharedArray{Float64}(101, 3);
    
@distributed for i in 1:101
    popperBoot[i, :] = [Bootstrap.confint(bs(percentages[:, i, 4]), BasicConfInt(.95))[1]...]
end;

In [ ]:
Generation = repeat(collect(1:101), outer=4)
Rule = repeat(["Bayes", "EXPL", "Good", "Popper"], inner=101)
type_df = convert(DataFrame, hcat(vcat(bayesBoot, explBoot, goodBoot, popperBoot), Generation, Rule))
rename!(type_df, [Symbol("$i") for i in ["y", "ymin", "ymax", "Generation", "Rule"]]);

In [ ]:
plot(type_df, x=:Generation, y=:y, ymin=:ymin, ymax=:ymax, color=:Rule, Geom.line, Geom.ribbon,
    Guide.ylabel("Average percentage"),
    Coord.cartesian(xmin=1, xmax=101, ymin=-.001),
    Scale.color_discrete_manual(gen_brew_colors(4)...),
    style(line_width=2pt, lowlight_color=c->RGBA{Float32}(c.r, c.g, c.b, 0.2),
            minor_label_font_size=10pt, major_label_font_size=14pt,
            key_label_font_size=11pt, key_title_font_size=13pt,
            colorkey_swatch_shape=:square))

We do the same for the bonus values.

In [ ]:
full_results_c = Array{Float32, 3}(undef, 200, 101, 100)

for i in 1:100
    full_results_c[:, :, i] = readdlm("data/weib_c_value$i.txt")
end

In [ ]:
expl_c_res = Array{Float64, 2}(undef, 101, 3)

for k in 1:101
    cr = Float64[]
    for j in 1:100
        push!(cr, mean([ full_results_c[:, k, i][(full_results_type[:, k, i] .== 2)] for i in 1:100 ][j]))
    end
    cr = cr[.!isnan.(cr)]
    expl_c_res[k, :] = vcat(mean(cr), [confint(OneSampleTTest(cr))...])
end

good_c_res = Array{Float64, 2}(undef, 101, 3)

for k in 1:101
    cr = Float64[]
    for j in 1:100
        push!(cr, mean([ full_results_c[:, k, i][(full_results_type[:, k, i] .== 3)] for i in 1:100 ][j]))
    end
    cr = cr[.!isnan.(cr)]
    good_c_res[k, :] = vcat(mean(cr), [confint(OneSampleTTest(cr))...])
end

pop_c_res = Array{Float64, 2}(undef, 101, 3)

for k in 1:101
    cr = Float64[]
    for j in 1:100
        push!(cr, mean([ full_results_c[:, k, i][(full_results_type[:, k, i] .== 4)] for i in 1:100 ][j]))
    end
    cr = cr[.!isnan.(cr)]
    pop_c_res[k, :] = vcat(mean(cr), [confint(OneSampleTTest(cr))...])
end

In [ ]:
Generation = repeat(collect(1:101), outer=3)
Rule = repeat(["EXPL", "Good", "Popper"], inner=101)
cval_df = convert(DataFrame, hcat(vcat(expl_c_res, good_c_res, pop_c_res), Generation, Rule))
rename!(cval_df, [Symbol("$i") for i in ["y", "ymin", "ymax", "Generation", "Rule"]]);

In [ ]:
plot(cval_df, x=:Generation, y=:y, ymin=:ymin, ymax=:ymax, color=:Rule, Geom.line, Geom.ribbon,
    Guide.ylabel("Average bonus value"),
    Coord.cartesian(xmin=1, xmax=101),
    Scale.color_discrete_manual(gen_brew_colors(4)[2:4]...),
    style(line_width=2pt, lowlight_color=c->RGBA{Float32}(c.r, c.g, c.b, 0.2),
            minor_label_font_size=10pt, major_label_font_size=14pt,
            key_label_font_size=11pt, key_title_font_size=13pt,
            colorkey_swatch_shape=:square))

Now the same for the fitness of the agents.

In [ ]:
full_results_f = Array{Float32, 3}(undef, 200, 101, 100)

for i in 1:100
    full_results_f[:, :, i] = readdlm("data/weib_fit$i.txt")
end

In [ ]:
bayes_f_res = Array{Float64, 2}(undef, 101, 3)

for k in 1:101
    cr = Float64[]
    for j in 1:100
        push!(cr, mean([ full_results_f[:, k, i][(full_results_type[:, k, i] .== 1)] for i in 1:100 ][j]))
    end
    cr = cr[.!isnan.(cr)]
    bayes_f_res[k, :] = length(unique(cr)) > 1 ? vcat(mean(cr), [confint(OneSampleTTest(cr))...]) : [NaN, NaN, NaN]
end

expl_f_res = Array{Float64, 2}(undef, 101, 3)

for k in 1:101
    cr = Float64[]
    for j in 1:100
        push!(cr, mean([ full_results_f[:, k, i][(full_results_type[:, k, i] .== 2)] for i in 1:100 ][j]))
    end
    cr = cr[.!isnan.(cr)]
    expl_f_res[k, :] = length(unique(cr)) > 1 ? vcat(mean(cr), [confint(OneSampleTTest(cr))...]) : [NaN, NaN, NaN]
end

good_f_res = Array{Float64, 2}(undef, 101, 3)

for k in 1:101
    cr = Float64[]
    for j in 1:100
        push!(cr, mean([ full_results_f[:, k, i][(full_results_type[:, k, i] .== 3)] for i in 1:100 ][j]))
    end
    cr = cr[.!isnan.(cr)]
    good_f_res[k, :] = length(unique(cr)) > 1 ? vcat(mean(cr), [confint(OneSampleTTest(cr))...]) : [NaN, NaN, NaN]
end

pop_f_res = Array{Float64, 2}(undef, 101, 3)

for k in 1:101
    cr = Float64[]
    for j in 1:100
        push!(cr, mean([ full_results_f[:, k, i][(full_results_type[:, k, i] .== 4)] for i in 1:100 ][j]))
    end
    cr = cr[.!isnan.(cr)]
    pop_f_res[k, :] = length(unique(cr)) > 1 ? vcat(mean(cr), [confint(OneSampleTTest(cr))...]) : [NaN, NaN, NaN]
end

In [ ]:
plot(
    layer(x=1:101,
    y=expl_f_res[1:101, 1],
    Geom.line,
    Theme(default_color=colorant"#fc8d62", line_width=2pt)
    ),
    layer(x=1:101,
    y=good_f_res[1:101, 1],
    Geom.line,
    Theme(default_color=colorant"#8da0cb", line_width=2pt)
    ),
    layer(x=1:101,
    y=pop_f_res[1:101, 1],
    Geom.line,
    Theme(default_color=colorant"#e78ac3", line_width=2pt)
    ),
    layer(x=1:length(bayes_f_res[.!isnan.(bayes_f_res[:, 1]), 1]),
    y=bayes_f_res[.!isnan.(bayes_f_res[:, 1]), 1],
    ymin=bayes_f_res[.!isnan.(bayes_f_res[:, 1]), 2],
    ymax=bayes_f_res[.!isnan.(bayes_f_res[:, 1]), 3],
    Geom.line,
    Geom.ribbon,
    Theme(default_color=colorant"#66c2a5", line_width=2pt, lowlight_color=c->RGBA{Float32}(c.r, c.g, c.b, 0.2))
    ),
    layer(x=1:101,
    y=expl_f_res[1:101, 1],
    ymin=expl_f_res[1:101, 2],
    ymax=expl_f_res[1:101, 3],
    Geom.line,
    Geom.ribbon,
    Theme(default_color=colorant"#fc8d62", lowlight_color=c->RGBA{Float32}(c.r, c.g, c.b, 0.2))
    ),
    layer(x=1:101,
    y=good_f_res[1:101, 1],
    ymin=good_f_res[1:101, 2],
    ymax=good_f_res[1:101, 3],
    Geom.line,
    Geom.ribbon,
    Theme(default_color=colorant"#8da0cb", lowlight_color=c->RGBA{Float32}(c.r, c.g, c.b, 0.2))
    ),
    layer(x=1:101,
    y=pop_f_res[1:101, 1],
    ymin=pop_f_res[1:101, 2],
    ymax=pop_f_res[1:101, 3],
    Geom.line,
    Geom.ribbon,
    Theme(default_color=colorant"#e78ac3", lowlight_color=c->RGBA{Float32}(c.r, c.g, c.b, 0.2))
    ),
    Guide.xlabel("Generation"),
    Guide.ylabel("Average fitness"),
    Coord.cartesian(xmax=101, ymin=.81),
    Guide.manual_color_key("Rule", ["Bayes", "EXPL", "Good", "Popper"], ["#66c2a5", "#fc8d62", "#8da0cb", "#e78ac3"]),
    style(minor_label_font_size=10pt, major_label_font_size=14pt,
            key_label_font_size=11pt, key_title_font_size=13pt,
            colorkey_swatch_shape=:square))

### Summary information about last generations

In [ ]:
last_gen_types = sum([ sum((full_results_type[:, 101, i] .== 1)) for i in 1:100 ]), sum([ sum((full_results_type[:, 101, i] .== 2)) for i in 1:100 ]), sum([ sum((full_results_type[:, 101, i] .== 3)) for i in 1:100 ]), sum([ sum((full_results_type[:, 101, i] .== 4)) for i in 1:100 ])

In [ ]:
plot(x = ["Bayes", "EXPL", "Good", "Popper"], y = [last_gen_types...], Geom.bar,
    Guide.xlabel("Rule"),
    Guide.ylabel("Count"),
    Scale.x_discrete,
    style(default_color=colorant"#66c2a5", minor_label_font_size=11pt, major_label_font_size=15pt,
            bar_spacing=35pt))

In [ ]:
lastE = Float64[]
for j in 1:100
    append!(lastE, [ full_results_c[:, 101, i][(full_results_type[:, 101, i] .== 2)] for i in 1:100 ][j])
end

lastG = Float64[]
for j in 1:100
    append!(lastG, [ full_results_c[:, 101, i][(full_results_type[:, 101, i] .== 3)] for i in 1:100 ][j])
end

lastP = Float64[]
for j in 1:100
    append!(lastP, [ full_results_c[:, 101, i][(full_results_type[:, 101, i] .== 4)] for i in 1:100 ][j])
end

In [ ]:
df = DataFrame(Rule = vcat(fill("EXPL", length(lastE)), fill("Good", length(lastG)), fill("Popper", length(lastP))), C_value = vcat(lastE, lastG, lastP));

In [ ]:
plot(df, x=:C_value, color=:Rule, Geom.density(bandwidth=.005),
     Coord.cartesian(xmin=-.0075, xmax=.261),
     Scale.color_discrete_manual(gen_brew_colors(4)[2:4]...),
     Guide.xlabel("Bonus value"),
     Guide.ylabel("Density"),
     style(line_width=2.65pt, minor_label_font_size=10pt, major_label_font_size=14pt,
            key_label_font_size=11pt, key_title_font_size=13pt,
            colorkey_swatch_shape=:square))

### Statistics

In [ ]:
using RCall

In [ ]:
lastEF = Float64[]
for j in 1:100
    append!(lastEF, [ full_results_f[:, 101, i][(full_results_type[:, 101, i] .== 2)] for i in 1:100 ][j])
end

lastGF = Float64[]
for j in 1:100
    append!(lastGF, [ full_results_f[:, 101, i][(full_results_type[:, 101, i] .== 3)] for i in 1:100 ][j])
end

lastPF = Float64[]
for j in 1:100
    append!(lastPF, [ full_results_f[:, 101, i][(full_results_type[:, 101, i] .== 4)] for i in 1:100 ][j])
end

In [ ]:
x = vcat(lastEF, lastGF, lastPF)
y = vcat(fill("EXPL", length(lastEF)), fill("Good", length(lastGF)), fill("Popper", length(lastPF)))
R"summary(aov($x ~ $y))"

In [ ]:
R"pairwise.t.test($x, $y, p.adj = 'bonf')"

In [ ]:
R"library(lsr)"
R"eSq <- etaSquared(aov($x ~ $y))[1]"

In [ ]:
R"library(emmeans)"
R"df <- data.frame(x = $x, y = $y)"
R"mod <- lm(x ~ y, data = df)"
R"emmeans(mod, ~ y)"

In [ ]:
x1 = vcat(lastE, lastG, lastP)
y1 = vcat(fill("EXPL", length(lastE)), fill("Good", length(lastG)), fill("Popper", length(lastP)))
R"df <- data.frame(x = $x1, y = $y1)"
R"mod <- lm(x ~ y, data = df)"
R"emmeans(mod, ~ y)"